In [ ]:
# import ipyleaflet as ipl 
# pridavali by sme postupne, ako by sa chyby objavovali
from ipyleaflet import Map, basemaps, basemap_to_tiles, LayersControl, ScaleControl, CircleMarker, Polyline, MarkerCluster
import gpxpy

In [ ]:
# zakladne vrstvy
# -------- Esri satelitna --------
esri_sat = basemap_to_tiles(basemaps.Esri.WorldImagery)
esri_sat.name = 'Esri Sat'
# -------------- OSM -------------
mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
mapnik.name = 'OSM Mapnik'
esri_sat.base = mapnik.base = True # dolezite
# zakladna mapa
m = Map(center=(49.05, 18.95), layers=[esri_sat, mapnik], zoom=13) # iny sposob, nie cez add_layer
m.layout.height = '600px'
lc =  LayersControl(position='topright')
m.add(lc)  # poznamka o add_control, add_layer, menach premennych pre vrstvy
sc = ScaleControl(imperial=False, position='bottomright')
m.add(sc);   # preco bodkociarka

In [ ]:
# ukazat trasy/hradiste.gpx, chceme ho nacitat
def read_gpx(gfile, meters=0):
    gpx_file = open(gfile, 'r')
    gpx = gpxpy.parse(gpx_file)  # vrati GPX objekt, nizsie ukazat, co vsetko obsahuje
    gpx.simplify(meters)         # vidime efekt dole
    lat, lon = [], []
    # assume one track, one segment, data: lat, lon, elev
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                lat.append(point.latitude)
                lon.append(point.longitude)    
    return (lat, lon)

In [ ]:
# TAB pre doplnenie
# gpxpy.gpx.GPX.

In [ ]:
lat, lon = read_gpx('trasy/hradiste.gpx')
len(lat)
slat, slon = read_gpx('trasy/hradiste.gpx', meters=2)
len(lat), len(slat)

In [ ]:
# Polyline?
loc = list(zip(lat, lon))
sloc = list(zip(slat, slon))
ori_track = Polyline(locations=loc, fill=False, weight=2, name='Pôvodná trasa')
sim_track = Polyline(locations=sloc, fill=False, weight=2, color='cyan', name='Zjednoduš. trasa')
m.add(ori_track)
m.add(sim_track);

In [ ]:
m

In [ ]:
# aby trasa bola 'vycentrovana'
npts = len(slat)
m.center = (sum(slat) / npts, sum(slon) / npts)

In [ ]:
# Zobrazenie bodov trasy, frajerske
points = []
for la, lo in sloc:
    point = CircleMarker(location=(la, lo), radius=3, color='red')
    points.append(point)
    # m.add_layer(point) # NEROBTE, pretazime stroj
m.add(MarkerCluster(markers=points)); 

__Našim cieľom je zobraziť len body, kde sme na trase fotili a po kliknutí na bod aj miniatúru fotky.__